In [18]:
# login
%load_ext autoreload
%autoreload 2

from org.kepco import Kepco
import agent
import logger

logger.logger_init(basedir=agent.iaa_get_config_directory())

k = Kepco('jaeyoungs4585', "Wodudqkr@845", "GetLastError#2")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
open logger.json


In [24]:
# login
%load_ext autoreload
%autoreload 2

from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.by import By
from org.kepco import *
import os
import time

class Temp:
    def __init__(self, number):
        self.number = number

driver = k.driver
number = "G012202823"

# def kepco_pre_register(driver: WebDriver, number):

# 1. 공고번호 조회

# 1.1 입찰 참가신청 페이지로 이동
kepco_pre_go_register_page(driver)


kepco_pre_search_notice_number(driver, number)




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
kepco_pre_validate_notice_number(driver)

True

In [ ]:
# 1.2 조회

#  1.3
# log().info("1.3 validate")
kepco.kepco_pre_validate_notice_number(driver):
    raise Exception("F")
    # log().info("Already registered")
    # return True

# log().info("1.4 apply")
org.kepco.kepco_pre_apply_participation(driver)

# 2. Application Form
# 2.1.  메세지등의 팝업을 처리한다.
org.kepco.kepco_pre_close_popup(driver)

# 2.2. 중소기업확인서 첨부
filepath = os.path.join(os.path.expanduser("~"), ".iaa", "AR_중소기업_확인서.pdf")
org.kepco.kepco_attach_small_business_confirmation_document(driver, filepath)

# XXX: 빠르게 진행하기 때문에 다시 제출하라는 문구가 뜨는 것 같음
time.sleep(5)

# 모든 조항에 동의
org.kepco.kepco_pre_agree_commitments(driver)

# XXX: 빠르게 진행하기 때문에 다시 제출하라는 문구가 뜨는 것 같음
time.sleep(5)

# 2.3 제출 버튼 클릭
org.kepco.kepco_pre_submit_application_form(driver)

# 3. 정리
# 3.1 확인
org.kepco.kepco_pre_confirm_submission(driver)

# 3.2 certificate login
org.kepco.kepco_certificate_login(driver, "GetLastError#2")

# 3.3 confirm done
org.kepco.kepco_pre_confirm_done(driver)